In [5]:
PINECONE_API_KEY = ''
GOOGLE_API_KEY = ''

In [8]:
import sqlite3
import json
from pinecone import Pinecone, ServerlessSpec
import time

import google.generativeai as genai
import os


pc = Pinecone(api_key=PINECONE_API_KEY)

# Create or connect to an index
index_name = "my-store-products"
if index_name not in pc.list_indexes():
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

genai.configure(api_key=GOOGLE_API_KEY)

conn = sqlite3.connect("C:/Users/Ervin/Desktop/Projects/vue-basics/laravel-api/database/database.sqlite")
cursor = conn.cursor()

cursor.execute("SELECT id, name, description, price, image FROM products")
products = cursor.fetchall()

for product in products:
    product_id, name, description, price, image = product

    # Create JSON representation
    product_json = json.dumps({
        "id": product_id,
        "name": name,
        "description": description,
        "price": price,
        "image": image
    }, ensure_ascii=False)

    response = genai.embed_content(model="models/text-embedding-004", content=product_json, task_type="retrieval_document")
    embedding = response["embedding"]

    index.upsert([(str(product_id), embedding, {"json": product_json})])

print("Products successfully inserted into Pinecone!")


Products successfully inserted into Pinecone!


In [9]:
import json
import google.generativeai as genai
from pinecone import Pinecone

# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "my-store-products"
index = pc.Index(index_name)

# Initialize Google Generative AI
genai.configure(api_key=GOOGLE_API_KEY)

def retrieve_products(query: str, top_k: int = 20):
    # Generate embedding for query
    response = genai.embed_content(model="models/text-embedding-004", content=query, task_type="retrieval_query")
    query_embedding = response["embedding"]

    # Search in Pinecone
    results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)

    # Extract and return product details
    products = [json.loads(match["metadata"]["json"]) for match in results["matches"]]
    return products

# Example usage
query = "Men's stylish casual jacket for cold weather"
retrieved_products = retrieve_products(query)

# Print retrieved products
for product in retrieved_products:
    print(f"Name: {product['name']}, Price: PHP {product['price']}, Description: {product['description']}, Image: {product['image']}")


Name: Peacoat, Price: PHP 2800, Description: This peacoat is a timeless outerwear choice that never goes out of style. Crafted from high-quality wool, it’s perfect for chilly weather and adds a polished look to any outfit. (Men), Image: images/products/Peacoat.jpg
Name: Quilted Jacket, Price: PHP 2200, Description: This quilted jacket offers lightweight warmth with a stylish design. Perfect for transitional weather, it adds a chic touch to any outfit while keeping you cozy and protected from the cold. (Unisex), Image: images/products/Quilted Jacket.jpg
Name: Men Bomber Jacket, Price: PHP 2000, Description: This men’s bomber jacket is the perfect blend of style and function. Designed for those who appreciate both fashion and comfort, it features a sleek design, ribbed cuffs, and a versatile look that can be paired with almost anything. (Men), Image: images/products/Men Bomber Jacket.jpg
Name: Stylish Leather Jacket, Price: PHP 3500, Description: Elevate your wardrobe with this premium l